## Brazilian E-Commerce Site Public Dataset by Olist
## Introduction

This is a Brazilian ecommerce public dataset of orders made at Olist Store. The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil. Its features allows viewing an order from multiple dimensions: from order status, price, payment and freight performance to customer location, product attributes and finally reviews written by customers. It also contains a geolocation dataset of Brazilian zip codes 

## Steps
1. Import packages and raw datasets
2. Create the database schema
3. Normalize Database
4. Transform data to fit purpose
5. Connect to the sqlserver database 
6. Load data into database

## 1. Libraries

In [2]:
import numpy as np 
import pandas as pd 
import pyodbc 
import sys
from datetime import date
import string

## 2. Reading The Data

In [6]:
#import datasets 
customers = pd.read_csv('olist_customers_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
product_category = pd.read_csv('product_category_name_translation.csv')

## 2.1 Extract Data

This dataset was provided by Olist, the largest department store in Brazilian marketplaces to Kaggle.com. Olist connects small businesses from all over Brazil to channels without hassle and with a single contract. Those merchants are able to sell their products through the Olist Store and ship them directly to the customers using Olist logistics partners. 

The extraction part of this project was a simple download of all required datasets from Kaggle.com 

# 3. Transform (Feature Engineer Data)

## 3.1. Geolocation - Address

Lets Construct the "address" table as designed in the database schema. This table will have the tables listed below:
1. geolocation_id	
2. zip_code_prefix	
3. geolocation_city	
4. geolocation_state

The geolocation_id will serve as the Primary Key for this table.

In [7]:
#View dataset
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [8]:
geo_df = geolocation.drop_duplicates() #drop duplicated rows to get the unique rows
len(geo_df)

738332

In [9]:
geo_data = geo_df[['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state']].copy()

In [10]:
#rename the columns to make it consistent with the "address" table
geo_data.columns = ['zip_code_prefix', 'geolocation_city', 'geolocation_state']

In [11]:
geo_data.drop_duplicates(['zip_code_prefix'],inplace = True) #drop duplicates

In [12]:
#construct the "geolocation_id"
geo_data.insert(0, 'geolocation_id', range(1, 1+len(geo_data)))

In [13]:
#change the order of the columns to make it consistent with the "address" table
geo_data = geo_data[['geolocation_id','zip_code_prefix', 'geolocation_city', 'geolocation_state']]

In [14]:
geo_data.reset_index(drop=True,inplace= True) #Reset dataframe Index

In [15]:
#Remove Unicode characters from geolocation_city
punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{}~'  
transtab = str.maketrans(dict.fromkeys(punct, ''))
 
geo_data['geolocation_city'] = '|'.join(geo_data['geolocation_city'].tolist()).translate(transtab).split('|')

In [16]:
geo_data.head()

,geolocation_id,zip_code_prefix,geolocation_city,geolocation_state
0,1,1037,sao paulo,SP
1,2,1046,sao paulo,SP
2,3,1041,sao paulo,SP
3,4,1035,sao paulo,SP
4,5,1012,são paulo,SP


## 3.2. Customers

Lets Construct the "customers" table as designed in the database schema. This table will have the tables listed below:
1. customer_id	
2. customer_unique_id	
3. geolocation_id

The customer_id will serve as the Primary Key for this table.

In [17]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [18]:
#Merge two tables together by zipcode using Inner Join.
temp_customers = customers.merge(geo_data, left_on= customers['customer_zip_code_prefix'], 
                                 right_on = geo_data['zip_code_prefix'], how = 'inner')

In [19]:
#Pick the 3 variables for this table
customers_df = temp_customers[['customer_id', 'customer_unique_id', 'geolocation_id']]

In [20]:
customers_df = customers_df.drop_duplicates() #drop duplicate rows
len(customers_df)

99163

In [21]:
customers_df.head()

,customer_id,customer_unique_id,geolocation_id
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,5353
1,5dca924cc99eea2dc5ba40d11ec5dd0f,2761fee7f378f0a8d7682d8a3fa07ab1,5353
2,661897d4968f1b59bfff74c7eb2eb4fc,d06a495406b79cb8203ea21cc0942f8c,5353
3,702b62324327ccba20f1be3465426437,8b3d988f330c1d1c3332ccd440c147b7,5353
4,bdf997bae7ca819b0415f5174d6b4302,866755e25db620f8d7e81b351a15bb2f,5353


## 3.3. Sellers

Lets Construct the "sellers" table as designed in the database schema. This table will have the tables listed below:
1. seller_id	
2. geolocation_id

The seller_id will serve as the Primary Key for this table.

In [22]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [23]:
#Merge two tables together by zipcode using Inner Join.
temp_sellers = sellers.merge(geo_data, left_on= sellers['seller_zip_code_prefix'], 
                             right_on = geo_data['zip_code_prefix'], how = 'inner')

In [24]:
#pick two variables
sellers_df = temp_sellers[['seller_id', 'geolocation_id']]
sellers_df['geolocation_id'].unique()

array([ 4806,  5197,  6358, ...,  2169, 18505,  4570])

In [25]:
sellers_df = sellers_df.drop_duplicates() #drop duplicate rows
len(sellers_df)

3088

In [26]:
sellers_df.head()

,seller_id,geolocation_id
0,3442f8959a84dea7ee197c632cb2df15,4806
1,e0eabded302882513ced4ea3eb0c7059,4806
2,d1b65fc7debc3361ea86b5f14c68d2e2,5197
3,ce3ad9de960102d0677a81f5d0bb7b2d,6358
4,1d2732ef8321502ee8488e8bed1ab8cd,6358


## 3.4. Product Category

Lets Construct the "product_category" table as designed in the database schema. This table will have the tables listed below:
1. product_category_id	
2. product_category_name_english

The product_category_id will serve as the Primary Key for this table.

In [27]:
product_category.head() #each row only contains one item (e.g. "health_beauty" means one category)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [28]:
category_df = product_category.drop_duplicates() #drop duplicate rows
len(category_df)

71

In [29]:
#pick the id and english name column and save it as a new dataframe
category = category_df[['product_category_name_english']] 
category.head() 

,product_category_name_english
0,health_beauty
1,computers_accessories
2,auto
3,bed_bath_table
4,furniture_decor


In [30]:
category.insert(0, 'product_category_id', range(1, 1+len(category))) #construct the id column for product category

In [31]:
category.head()

,product_category_id,product_category_name_english
0,1,health_beauty
1,2,computers_accessories
2,3,auto
3,4,bed_bath_table
4,5,furniture_decor


## 3.5. Products

Lets Construct the "product" table as designed in the database schema. This table will have the tables listed below:
1. product_id	
2. product_name_length	
3. product_description_length	
4. product_photos_qty	
5. product_category_id	
6. product_weight	
7. product_length	
8. product_height	
9. product_width	


The product_id will serve as the Primary Key for this table.

In [32]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [33]:
#merge the original product category dataset and products datasets by product category name
products_temp = products.merge(category_df, left_on = products['product_category_name'], 
                             right_on = category_df['product_category_name'], how = 'left') 


In [34]:
products_temp.drop('key_0', axis=1, inplace=True)

In [35]:
#merge the datasets to get the product_category_id
products_df = products_temp.merge(category, left_on = products_temp['product_category_name_english'], 
                                 right_on = category['product_category_name_english'], how = 'inner')

In [36]:
#drop unnecessary columns
products_df.drop(['key_0','product_category_name_x','product_category_name_y', 'product_category_name_english_x', 'product_category_name_english_y'],axis = 1, inplace = True)

In [37]:
#rename the columns
products_df.columns = ['product_id', 'product_name_length', 'product_description_length', 'product_photos_qty',
                       'product_weight', 'product_length', 'product_height', 'product_width', 'product_category_id']

In [38]:
#drop duplicates and save the results in a new dataframe
products_df_2 = products_df.drop_duplicates()

In [39]:
#change the order
products_df_2 = products_df_2[['product_id', 'product_name_length', 'product_description_length', 
                               'product_photos_qty','product_category_id', 
                               'product_weight', 'product_length', 'product_height', 'product_width']]

In [40]:
#change data type from float to integer
for name in ['product_name_length', 'product_description_length', 'product_photos_qty']:
    products_df_2[name] = products_df_2[name].apply(lambda x:int(x))

In [41]:
products_df_2.head()

,product_id,product_name_length,product_description_length,product_photos_qty,product_category_id,product_weight,product_length,product_height,product_width
0,1e9e8ef04dbcff4541ed26657ea517e5,40,287,1,7,225.0,16.0,10.0,14.0
1,6a2fb4dd53d2cdb88e0432f1284a004c,39,346,2,7,400.0,27.0,5.0,20.0
2,0d009643171aee696f4733340bc2fdd0,52,150,1,7,422.0,21.0,16.0,18.0
3,b1eae565a61935e0011ee7682fef9dc9,49,460,2,7,267.0,17.0,13.0,17.0
4,8da90b37f0fb171b4877c124f965b1f6,56,733,3,7,377.0,18.0,13.0,15.0


## 3.6. Orders

Lets Construct the "orders" table as designed in the database schema. This table will have the tables listed below:
1. order_id	
2. customer_id	
3. order_purchase_timestamp		

The order_id will serve as the Primary Key for this table.

It is critical to point out that one order might include multiple products. Orders and Products tables have a many-to-many relationship. In order to reduce the database redundancy, we construct a "products_ordered" table including the order_id and product_id, which helps link orders and products table together. 

In [42]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [43]:
order = orders.drop_duplicates() #drop the duplicates and check the number of duplicates

In [44]:
order_items.head() #we need to check another dataset "order_items"

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [45]:
order_items = order_items.drop_duplicates() #drop duplicate entries

In [46]:
#Merge order dataset with order_items dataset by order_id
order = order_items.merge(orders, left_on = order_items['order_id'], 
                          right_on = orders['order_id'], 
                          how = 'inner')

In [47]:
#drop unnecessary columns that will be included in the delivery table
order.drop(['key_0','order_status', 'order_id_y','order_approved_at', 'order_delivered_carrier_date', 
                   'order_estimated_delivery_date', 'order_delivered_customer_date'], axis = 1, inplace = True)

In [48]:
#reorder the dataframe columns and save it into a new order dataframe
order_df = order[['order_id_x', 'customer_id', 'order_purchase_timestamp']].copy()

In [49]:
#rename the columns
order_df.columns = ['order_id', 'customer_id', 'order_purchase_timestamp']

In [50]:
#Drop the duplicated rows in the new order dataframe
order_df_2 = order_df.drop_duplicates()

In [51]:
#Since customer_id is a foreign key, we need to merge the order table with customers table to get the intersection
order_new = order_df_2.merge(customers_df, left_on = order_df_2['customer_id'], 
                          right_on = customers_df['customer_id'], 
                          how = 'inner')

In [52]:
#drop unnecessary columns that will be included in the delivery table
order_new.drop(['key_0','customer_unique_id', 'customer_id_y','geolocation_id'], axis = 1, inplace = True)

In [53]:
order_new.columns = ['order_id', 'customer_id', 'order_purchase_timestamp'] #rename the columns
order_new.head()

,order_id,customer_id,order_purchase_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51


## 3.7. Delivery

Lets Construct the "delivery" table as designed in the database schema. This table will have the tables listed below:
1. delivery_id	
2. order_id	
3. order_status	
4. order_approved_at	
5. order_delivered_carrier_date	
6. order_estimated_delivery_date	
7. order_delivered_customer_date
	

The delivery_id	will serve as the Primary Key for this table.

The delivery information is all stored in the "orders" table. By isolating the required columns, the delivery table can then be constructed. Duplicates would also need to be removed and a "delivery_id" column added. 

In [54]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [55]:
#pick the variables needed from orders
delivery = orders[['order_id', 'order_status', 'order_approved_at', 'order_delivered_carrier_date', 
                   'order_estimated_delivery_date', 'order_delivered_customer_date']]

In [56]:
delivery_t = delivery.drop_duplicates() #drop duplicate entries

In [57]:
#Since order_id is the foreign key, we need to merge the delivery dataset with order dataset to get the intersection.
delivery_t = delivery_t.merge(order_new, left_on = delivery_t['order_id'], 
                          right_on = order_new['order_id'], 
                          how = 'inner')

In [58]:
#drop unnecessary columns and reorder the dataset after merging
delivery_t = delivery_t[['order_id_x', 'order_status', 'order_approved_at', 'order_delivered_carrier_date', 
                   'order_estimated_delivery_date', 'order_delivered_customer_date']]

In [59]:
#rename the columns
delivery_t.columns = ['order_id', 'order_status', 'order_approved_at', 'order_delivered_carrier_date', 
                   'order_estimated_delivery_date', 'order_delivered_customer_date']

In [60]:
delivery_t.insert(0, 'delivery_id', range(1, 1+len(delivery_t))) #construct the delivery_id

In [61]:
delivery_t  = delivery_t.dropna(axis=0) #Remove rows with null values
delivery_t.head()

,delivery_id,order_id,order_status,order_approved_at,order_delivered_carrier_date,order_estimated_delivery_date,order_delivered_customer_date
0,1,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-18 00:00:00,2017-10-10 21:25:13
1,2,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-13 00:00:00,2018-08-07 15:27:45
2,3,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-09-04 00:00:00,2018-08-17 18:06:29
3,4,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-15 00:00:00,2017-12-02 00:28:42
4,5,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-26 00:00:00,2018-02-16 18:17:02


## 3.8. Products_ordered

Lets Construct the "products_ordered" table as designed in the database schema. This table will have the tables listed below:
1. pro_ordered_id	
2. order_id	
3. product_id	
4. seller_id	
5. item_price	
6. order_freight_value	

	

The pro_ordered_id	will serve as the Primary Key for this table.

This products_ordered table specifies the order and product relationship by selecting the relavant columns from the order dataframe.

In [62]:
#selecting the variables from the order dataframe and save it as the products_ordered dataframe
products_ordered = order[['order_id_x', 'product_id', 'seller_id', 'price', 'freight_value']]
products_ordered.columns = ['order_id', 'product_id', 'seller_id', 'item_price', 'order_freight_value']

In [63]:
products_ordered_df = products_ordered.drop_duplicates() #drop duplicated rows

In [64]:
products_ordered.head()

,order_id,product_id,seller_id,item_price,order_freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14


In [65]:
#Since order_id is the foreign key, we need to merge the products_ordered dataset with order dataset to get the intersection.
pro_ordered_df = products_ordered.merge(order_new, left_on = products_ordered['order_id'], 
                          right_on = order_new['order_id'], 
                          how = 'inner')

In [66]:
#drop unnecessary columns 
pro_ordered_df.drop(['key_0','order_id_y', 'customer_id','order_purchase_timestamp'], axis = 1, inplace = True)

In [67]:
#rename the columns
pro_ordered_df.columns = ['order_id', 'product_id', 'seller_id', 'item_price', 'order_freight_value']

In [68]:
#Since product_id is the foreign key, we need to merge the products_ordered dataset with order dataset to get the intersection.
pro_ordered_df_2 = pro_ordered_df.merge(products_df_2, left_on = pro_ordered_df['product_id'], 
                          right_on = products_df_2['product_id'], 
                          how = 'inner')

In [69]:
#drop unnecessary columns 
pro_ordered_df_2 = pro_ordered_df_2[['order_id', 'product_id_x', 'seller_id', 'item_price', 'order_freight_value']]

In [70]:
#rename the columns
pro_ordered_df_2.columns = ['order_id', 'product_id', 'seller_id', 'item_price', 'order_freight_value']

In [71]:
#Since seller_id is the foreign key, we need to merge the product_ordered dataset with order dataset to get the intersection.
pro_ordered_df_3 = pro_ordered_df_2.merge(sellers_df, left_on = pro_ordered_df_2['seller_id'], 
                          right_on = sellers_df['seller_id'], 
                          how = 'inner')

In [72]:
#drop unnecessary columns
pro_ordered_df_3 = pro_ordered_df_3[['order_id', 'product_id', 'seller_id_x', 'item_price', 'order_freight_value']]

In [73]:
#rename the columns
pro_ordered_df_3.columns = ['order_id', 'product_id', 'seller_id', 'item_price', 'order_freight_value']

In [74]:
pro_ordered_df_3 = pro_ordered_df_3.drop_duplicates() #drop duplicate entries

In [75]:
pro_ordered_df_3.insert(0, 'pro_ordered_id', range(1, 1+len(pro_ordered_df_3))) #construct the pro_ordered_id

In [76]:
pro_ordered_df_3.head()

,pro_ordered_id,order_id,product_id,seller_id,item_price,order_freight_value
0,1,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,13.29
1,2,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,55.9,17.96
2,3,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,64.9,18.33
3,4,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,16.17
4,5,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,13.29


## 3.9. Reviews

Lets Construct the "reviews" table as designed in the database schema. This table will have the tables listed below:
1. review_order_id	
2. review_id	
3. order_id	
4. review_score	
5. review_creation_date	
6. review_answer_timestamp


The review_order_id	will serve as the Primary Key for this table.

In [77]:
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [78]:
#Since order_id is the foreign key, we need to merge the reviews dataset with order dataset to get the intersection.
reviews_df = reviews.merge(order_new, left_on = reviews['order_id'], 
                          right_on = order_new['order_id'], 
                          how = 'inner')

In [79]:
#drop unnecessary columns
reviews_df = reviews_df[['review_id', 'order_id_x', 'review_score','review_creation_date', 'review_answer_timestamp']]

In [80]:
#rename the columns
reviews_df.columns = ['review_id', 'order_id', 'review_score', 'review_creation_date', 'review_answer_timestamp']

In [81]:
reviews_df.insert(0, 'review_order_id', range(1, 1+len(reviews_df))) #construct the review_order_id

In [82]:
reviews_df = reviews_df.drop_duplicates() #drop duplicate rows

In [83]:
reviews_df.head()

,review_order_id,review_id,order_id,review_score,review_creation_date,review_answer_timestamp
0,1,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,2018-01-18 00:00:00,2018-01-18 21:46:59
1,2,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,2018-03-10 00:00:00,2018-03-11 03:05:13
2,3,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,2018-02-17 00:00:00,2018-02-18 14:36:24
3,4,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,2017-04-21 00:00:00,2017-04-21 22:02:06
4,5,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,2018-03-01 00:00:00,2018-03-02 10:26:53


## 3.10. Payment

Lets Construct the "payment" table as designed in the database schema. This table will have the tables listed below:
1. payment_id	
2. order_id	
3. installments	
4. sequential	
5. payment_type	
6. transaction_value



The payment_id will serve as the Primary Key for this table.

In [84]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [85]:
payment = payments.drop_duplicates() #drop duplicates and check the dataframe

In [86]:
#Since order_id is the foreign key, we need to merge the payment dataset with order dataset to get the intersection.
payment_df = payment.merge(order_new, left_on = payment['order_id'], 
                          right_on = order_new['order_id'], 
                          how = 'inner')

In [87]:
#drop unnecessary columns
payment_df = payment_df[['order_id_x', 'payment_sequential', 'payment_type', 
                         'payment_installments', 'payment_value']] 

In [88]:
#rename the columns
payment_df.columns = ['order_id', 'payment_sequential', 'payment_type', 
                         'payment_installments', 'payment_value'] 

In [89]:
#change the order of the columns
payment_df = payment_df[['order_id', 'payment_installments', 'payment_sequential','payment_type', 'payment_value']]

In [90]:
#construct the payment_id
payment_df.insert(0, 'payment_id', range(1, 1+len(payment_df)))

In [91]:
#rename the columns
payment_df.columns = ['payment_id','order_id',
                    'installments','sequential',
                    'payment_type', 'transaction_value']

In [92]:
payment_df = payment_df.drop_duplicates() #drop duplicate rows
payment_df.head()

,payment_id,order_id,installments,sequential,payment_type,transaction_value
0,1,b81ef226f3fe1789b1e8b2acac839d17,8,1,credit_card,99.33
1,2,a9810da82917af2d9aefd1278f1dcfa0,1,1,credit_card,24.39
2,3,25e8ea4e93396b6fa0d3dd708e76c1bd,1,1,credit_card,65.71
3,4,ba78997921bbcdc1373bb41e913ab953,8,1,credit_card,107.78
4,5,42fdf880ba16b47b59251dd489d4441a,2,1,credit_card,128.45


## 3.11. Fact Table

Lets Construct the "fact" table as designed in the database schema. This table will have the tables listed below:
1. payment_id	
2. order_id	
3. geolocation_id	
4. customer_id
5. delivery_id	
6. product_id
7. product_category_id
8. product_ordered_id
9. review_order_id
10. sellers_id
11. fact_id

The fact_id will serve as the Primary Key for this table.

In [115]:
df1 = geo_data['geolocation_id']
df2= customers_df['customer_id']
df3 = sellers_df['seller_id']
df4= category['product_category_id']
df5= products_df_2['product_id']
df6= order_new['order_id']
df7= pro_ordered_df_3['pro_ordered_id']
df8= delivery_t['delivery_id']
df9= reviews_df['review_order_id']
df10= payment_df['payment_id']


In [116]:
fact_df=pd.DataFrame(columns=['geolocation_id','customer_id','seller_id','product_category_id','product_id','order_id','pro_ordered_id','delivery_id','review_order_id','payment_id'])

In [118]:
fact_df['geolocation_id']= df1
fact_df['customer_id']= df2
fact_df['seller_id']= df3
fact_df['product_category_id']= df4
fact_df['product_id']= df5
fact_df['order_id']= df6
fact_df['pro_ordered_id']= df7
fact_df['delivery_id']= df8
fact_df['review_order_id']= df9
fact_df['payment_id']= df10

In [127]:
fact_df

,geolocation_id,customer_id,seller_id,product_category_id,product_id,order_id,pro_ordered_id,delivery_id,review_order_id,payment_id
0,1,06b8999e2fba1a1fbc88172c00ba8bc7,3442f8959a84dea7ee197c632cb2df15,1.0,1e9e8ef04dbcff4541ed26657ea517e5,00010242fe8c5a6d1ba2dd792cb16214,1.0,1.0,1,1
1,2,5dca924cc99eea2dc5ba40d11ec5dd0f,e0eabded302882513ced4ea3eb0c7059,2.0,6a2fb4dd53d2cdb88e0432f1284a004c,00018f77f2f0320c557190d7a144bdd3,2.0,2.0,2,2
2,3,661897d4968f1b59bfff74c7eb2eb4fc,d1b65fc7debc3361ea86b5f14c68d2e2,3.0,0d009643171aee696f4733340bc2fdd0,000229ec398224ef6ca0657da4fc703e,3.0,3.0,3,3
3,4,702b62324327ccba20f1be3465426437,ce3ad9de960102d0677a81f5d0bb7b2d,4.0,b1eae565a61935e0011ee7682fef9dc9,00024acbcdf0a6daa1e931b038114c75,4.0,4.0,4,4
4,5,bdf997bae7ca819b0415f5174d6b4302,1d2732ef8321502ee8488e8bed1ab8cd,5.0,8da90b37f0fb171b4877c124f965b1f6,00042b26cf59d7ce69dfabb4e55b4fd9,5.0,5.0,5,5
...,...,...,...,...,...,...,...,...,...,...
19010,19011,d3fe25b67cddc1a46644b30c693cde08,NaN,NaN,feba83b90adfde26a6a944dee721e573,31b2ff78299f9c71d536cca98394ebe1,17221.0,19011.0,19011,19011
19011,19012,58f7e662bb4f4196f1496f7428b5cb46,NaN,NaN,fa0bace68b883626200a9047a5ea10ca,31b39339d93a6949e2f2addf798d8904,NaN,19012.0,19012,19012
19012,19013,355de5b8c9f5ca3b0b7484479c376137,NaN,NaN,35b1777d7b1b0e7839a9d0bd138f4c76,31b398923275e2f66a56071ac111efd5,17222.0,19013.0,19013,19013
19013,19014,d18bff6a3de67eebfab42468ffcc42a5,NaN,NaN,cdf516ee3c66d21bf05ed94c812a2642,31b3e1e5188e76431af2242ebbfb4ba4,17223.0,19014.0,19014,19014


## 4. Connect to SQLServer & Load Tables into Database

In [126]:
#Connect to SQL Server
try:
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-63K3API\SQLNICON;PORT=1433;DATABASE=OListDatabase;Trusted_Connection=yes;')

except Exception as e:
    print(e)
    print('Connection Failed. Task Terminated')
    sys.exit()

else:
    cursor = conn.cursor()

In [125]:
#Load the fact table into database
try:
        for index,data in fact_df.iterrows():
            cursor.execute("INSERT INTO dbo.id_table('geolocation_id','customer_id','seller_id','product_category_id','product_id','order_id','product_ordered_id','delivery_id','review_order_id','payment_id') VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                           data['geolocation_id'], data['customer_id'], data['seller_id'], data['product_category_id'],data['product_id'],data['order_id'],data['pro_ordered_id'],data['delivery_id'],data['review_order_id'],data['payment_id'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit()
        

('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'geolocation_id'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'customer_id'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'seller_id'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'product_category_id'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'product_id'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'order_id'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'product_ordered_id'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'delivery_id'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'review_order_id'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'payment

In [ ]:
#Connect to SQL Server
try:
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER=DESKTOP-63K3API\SQLNICON;PORT=1433;DATABASE=OListDatabase;Trusted_Connection=yes;')

except Exception as e:
    print(e)
    print('Connection Failed. Task Terminated')
    sys.exit()

else:
    cursor = conn.cursor()
    
    
    
#Load the address table into database
try:
        for index,data in geo_data.iterrows():
            cursor.execute("INSERT INTO dbo.address(geolocation_id,zip_code_prefix,geolocation_city,geolocation_state) VALUES (?, ?, ?, ?)",
                           data['geolocation_id'], data['zip_code_prefix'], data['geolocation_city'], data['geolocation_state'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit()
        

#Load the customers table into database
try:
        for index,data in customers_df.iterrows():
            cursor.execute("INSERT INTO dbo.customers(customer_id,customer_unique_id,geolocation_id) VALUES (?, ?, ?)",
                           data['customer_id'], data['customer_unique_id'], data['geolocation_id'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit()
        
        
#Load the sellers table into database
try:
        for index,data in sellers_df.iterrows():
            cursor.execute("INSERT INTO dbo.sellers(seller_id,geolocation_id) VALUES (?, ?)",
                           data['seller_id'], data['geolocation_id'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit()
        
        
#Load the product_category table into database
try:
        for index,data in category.iterrows():
            cursor.execute("INSERT INTO dbo.product_category(product_category_id,product_category_name_english) VALUES (?, ?)",
                           data['product_category_id'], data['product_category_name_english'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit()
        
#Load the products table into database
try:
        for index,data in products_df_2.iterrows():
            cursor.execute("INSERT INTO dbo.product(product_id,product_name_length,product_description_length,product_photos_qty,product_category_id,product_weight,product_length,product_height,product_width) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)",
                           data['product_id'], data['product_name_length'], data['product_description_length'], data['product_photos_qty'], data['product_category_id'], data['product_weight'], data['product_length'], data['product_height'], data['product_width'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit()
        
#Load the orders table into database
try:
        for index,data in order_new.iterrows():
            cursor.execute("INSERT INTO dbo.orders(order_id,customer_id,order_purchase_timestamp) VALUES (?, ?, ?)",
                           data['order_id'], data['customer_id'], data['order_purchase_timestamp'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit()
    
        
        
#Load the products_ordered table into database
try:
        for index,data in pro_ordered_df_3.iterrows():
            cursor.execute("INSERT INTO dbo.products_ordered (product_ordered_id, order_id, product_id, seller_id, item_price,order_freight_value) VALUES (?, ?, ?, ?, ?, ?)",
                       data['pro_ordered_id'],data['order_id'],data['product_id'],data['seller_id'],data['item_price'],data['order_freight_value'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit() 
        
#Load the delivery table into database
try:
        for index,data in delivery_t.iterrows():
            cursor.execute("INSERT INTO dbo.delivery(delivery_id,order_id,order_status,order_approved_at,order_delivered_carrier_date,order_estimated_delivery_date,order_delivered_customer_date) VALUES (?, ?, ?, ?, ?, ?, ?)",
                           data['delivery_id'],data['order_id'],data['order_status'],data['order_approved_at'],data['order_delivered_carrier_date'],data['order_estimated_delivery_date'], data['order_delivered_customer_date'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit()
        
        
#Load the reviews table into database
try:
        for index,data in reviews_df.iterrows():
            cursor.execute("INSERT INTO dbo.reviews(review_order_id, review_id, order_id, review_score, review_creation_date, review_answer_timestamp) VALUES (?, ?, ?, ?, ?, ?)",
                           data['review_order_id'],  data['review_id'],  data['order_id'],  data['review_score'], data['review_creation_date'], data['review_answer_timestamp'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit()
        
#Load the payment table into database
try:
        for index,data in payment_df.iterrows():
            cursor.execute("INSERT INTO dbo.payment(payment_id, order_id, installments, sequential, payment_type, transaction_value) VALUES (?, ?, ?, ?, ?, ?)",
                            data['payment_id'], data['order_id'], data['installments'], data['sequential'], data['payment_type'], data['transaction_value'])
        
except Exception as e:
        cursor.rollback()
        print (e)
        print('Transaction rolled back')
    
else:
        print('Records inserted Succesfully')
        cursor.commit()
        
finally:
        if conn:
            print('Connection Closed')
            conn.close()